In [ ]:
!nvidia-smi

Thu Nov  9 19:53:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai

In [ ]:
!pip install -qqq langchain==0.0.266
!pip install PyPDF2==2.12.1
!pip install -qqq sentence_transformers==2.2.2
!pip install -qqq InstructorEmbedding
!pip install -qqq pdf2image
!pip install pgvector
!pip install psycopg2
!pip install auto-gptq==0.4.2
!pip install -qqq transformers
!pip install -qqq sentence_transformers==2.2.2
!pip install -qqq torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:0

In [ ]:
import torch
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np
import json
import psycopg2
import pgvector
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.text_splitter import RecursiveCharacterTextSplitter
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.vectorstores.pgvector import PGVector

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
model1 = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:

conn = psycopg2.connect(
    host="*************",
    database="*******",
    user="*********",
    password="**********")

In [ ]:
cur = conn.cursor()
conn.rollback()
cur.execute("SELECT COUNT(*) as cnt FROM langchain_pg_embedding;")
num_records = cur.fetchone()[0]
print("Number of vector records in table: ", num_records,"\n")

Number of vector records in table:  4100 



In [ ]:
query = "parkinson disease"
query_embedding = model1.embed_query(query)

In [ ]:
query_embedding[0]

-0.01689971052110195

In [ ]:
def response_docs(query_embedding, conn):
    embedding_array = np.array(query_embedding)
    register_vector(conn)
    cur = conn.cursor()
    cur.execute("SELECT document FROM langchain_pg_embedding ORDER BY embedding <=> %s LIMIT 5", (embedding_array,))
    docs = cur.fetchall()
    return docs

In [ ]:
k=response_docs(query_embedding,conn)

In [ ]:
k

[("9//25//23, 9:14 PM Patient perspective: Parkinson disease - UpToDate\nhttps:////www.uptodate.com//contents//patient-perspective-parkinson-disease//print 1//10\nOfficial reprint from UpToDate\nwww.uptodate.com  2023 UpToDate, Inc. and//or its affiliates. All Rights Reserved.\nPatient perspective: Parkinson disease\nINTRODUCTION\nThis topic was written by an individual patient diagnosed with Parkinson disease (PD). It is\nintended to offer clinicians insight into the experience of a single individual from that\nindividual's point of view. This description of a particular patient's experience is not intended\nto be comprehensive or to provide recommendations regarding diagnosis, treatment, and//or\nmedication information. It is not intended to be medical advice or to be a substitute for the\nmedical advice, diagnosis, or treatment of a health care provider based on the health care\nprovider's examination and assessment of a patient's specific and unique circumstances.\nFor related clin

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"


tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. give answer for any questions.
 """.strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

System

In [ ]:
SYSTEM_PROMPT = "use the following system pieces of context to answer the questions. If you don't know the answer, just say you don't know, don't try to make up an answer."

template = generate_prompt("""
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
CONNECTION_STRING = "******************"
COLLECTION_NAME="sample_df"

store = PGVector(
    connection_string=CONNECTION_STRING,
    embedding_function=model1,
    collection_name=COLLECTION_NAME,

)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(search_kwargs={"k": 2}),
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
q=str(qa_chain("what is the parkinson disease"))
q

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Based on the provided context, I can answer the question as follows:

According to the patient perspective article from UpToDate, Parkinson disease (PD) is a neurodegenerative disorder that affects movement, balance, and coordination. It is characterized by the death of dopamine-producing neurons in a part of the brain called the substantia nigra, leading to motor symptoms such as tremors, rigidity, bradykinesia, and postural instability. The exact cause of PD is not known, but genetic and environmental factors are believed to play a role. There is no cure for PD, but various medications and therapies can help manage the symptoms and improve quality of life for patients.


"{'query': 'what is the parkinson disease', 'result': '  Based on the provided context, I can answer the question as follows:\\n\\nAccording to the patient perspective article from UpToDate, Parkinson disease (PD) is a neurodegenerative disorder that affects movement, balance, and coordination. It is characterized by the death of dopamine-producing neurons in a part of the brain called the substantia nigra, leading to motor symptoms such as tremors, rigidity, bradykinesia, and postural instability. The exact cause of PD is not known, but genetic and environmental factors are believed to play a role. There is no cure for PD, but various medications and therapies can help manage the symptoms and improve quality of life for patients.'}"

**Multi - Query With RAG**

In [ ]:
OPENAI_API_KEY ='********************'

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever = MultiQueryRetriever.from_llm(
    retriever= store.as_retriever(), llm=llm
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question = "what is the parkinson disease"

docs = retriever.get_relevant_documents(query=question)
len(docs)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. Can you provide information about Parkinson's disease?", "2. What are the symptoms and causes of Parkinson's disease?", "3. How does Parkinson's disease affect the body and what treatments are available?"]


3

In [ ]:
docs

[Document(page_content='provider\'s examination and assessment of a patient\'s specific and unique circumstances.\nFor related clinical topics, please see:\nUpToDate also offers patient education materials. "The Basics" topics are short overviews\nwritten in plain language, at the 5th to 6th grade reading level, that answer the main\nquestions a patient is likely to have about their condition. "Beyond the Basics" topics are\nwritten at the 10th to 12th grade reading level and are intended for readers who are seeking\nmore detail and are comfortable with some medical terminology. You can share this content\ndirectly with your patients:\n:Steven S Hamel\n :Kelvin L Chou, MD\n :April F Eichler, MD, MPH\nAll topics are updated as new evidence becomes available and our peer review process is complete.\nLiterature review current through:Aug 2023.\nThis topic last updated:Dec 07, 2022.\n(See "Clinical manifestations of Parkinson disease".) \n(See "Diagnosis and differential diagnosis of Parki

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [ ]:
out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in docs])
    }
)
out["text"]

"Parkinson's disease is a neurodegenerative disorder that affects movement. It is characterized by symptoms such as tremors, stiffness, slow movement, and difficulty with balance and coordination. The exact cause of Parkinson's disease is unknown, but it is believed to involve a combination of genetic and environmental factors. The disease progresses over time and can have a significant impact on a person's quality of life. Treatment options for Parkinson's disease include medication, physical therapy, and in some cases, surgery."